In [ ]:
# --- Kürzt ein Video auf die ersten n Frames ---
input_video = r"../videos/inputs/Altendorf_540p.mp4"
output_video = r"../videos/outputs/Altendorf_540p_short.mp4"
n_frames = 3600

# --- Logik (darunter nichts ändern) ---
import cv2
from pathlib import Path

def copy_first_n_frames(input_path, output_path, n):
    input_path = str(input_path)
    output_path = str(output_path)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise RuntimeError(f"Kann Eingabevideo nicht öffnen: {input_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if fps <= 0 or width <= 0 or height <= 0:
        cap.release()
        raise RuntimeError("Konnte Videoeigenschaften (FPS/Breite/Höhe) nicht bestimmen.")

    out_path = Path(output_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # gängiger MP4-Codec
    writer = cv2.VideoWriter(str(out_path), fourcc, fps, (width, height))
    if not writer.isOpened():
        cap.release()
        raise RuntimeError(f"Kann Ausgabedatei nicht schreiben: {out_path}")

    written = 0
    while written < n:
        ok, frame = cap.read()
        if not ok:
            break
        writer.write(frame)
        written += 1

    cap.release()
    writer.release()
    print(f"Fertig: {written} Frame(s) → '{out_path}' | FPS: {fps:.2f}, Auflösung: {width}x{height}")

copy_first_n_frames(input_video, output_video, n_frames)


Fertig: 3600 Frame(s) → '..\videos\outputs\Altendorf_540p_short.mp4' | FPS: 29.97, Auflösung: 960x540


In [3]:
input_video = r"../videos/outputs/Altendorf_540p_short.mp4"
output_video = r"../videos/outputs/Altendorf_540p_short_fast.mp4"
drop_every_k = 2
keep_every_k = 0
preserve_duration = False

import cv2
from pathlib import Path

def process_video(input_path, output_path, drop_every_k, keep_every_k, preserve_duration):
    if (drop_every_k > 0 and keep_every_k > 0) or (drop_every_k == 0 and keep_every_k == 0):
        print("Video kann nicht bearbeitet werden, weil genau einer der Parameter 'drop_every_k' oder 'keep_every_k' gesetzt sein muss.")
        return
    if drop_every_k == 1 or keep_every_k == 1:
        print("Video kann nicht bearbeitet werden, weil der gewählte Parameterwert 1 keinen Effekt hätte.")
        return

    cap = cv2.VideoCapture(str(input_path))
    if not cap.isOpened():
        print(f"Video kann nicht bearbeitet werden, weil es nicht geöffnet werden konnte: {input_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if fps <= 0 or w <= 0 or h <= 0:
        cap.release()
        print("Video kann nicht bearbeitet werden, weil die Eigenschaften (FPS/Breite/Höhe) ungültig sind.")
        return

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)

    if drop_every_k > 0:
        keep_func = lambda i: (i % drop_every_k) != 0
        ratio = (drop_every_k - 1) / drop_every_k
    else:
        keep_func = lambda i: (i % keep_every_k) == 0
        ratio = 1 / keep_every_k

    fps_out = max(fps * ratio, 1.0) if preserve_duration else fps

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(output_path), fourcc, fps_out, (w, h))
    if not writer.isOpened():
        cap.release()
        print(f"Video kann nicht bearbeitet werden, weil die Ausgabedatei nicht erstellt werden konnte: {output_path}")
        return

    i = kept = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        if keep_func(i):
            writer.write(frame)
            kept += 1
        i += 1

    cap.release()
    writer.release()
    print(f"Fertig: {kept} Frame(s) behalten → '{output_path}', FPS_in={fps:.2f}, FPS_out={fps_out:.2f}")

process_video(input_video, output_video, drop_every_k, keep_every_k, preserve_duration)


Fertig: 1800 Frame(s) behalten → '../videos/outputs/Altendorf_540p_short_fast.mp4', FPS_in=29.97, FPS_out=29.97
